In [27]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

In [28]:
with open("data/names.txt") as f:
    names = f.read().splitlines()

In [29]:
def build_ngram(s, n):
    res = []
    l = ['.'] + list(s) + ['.']
    for i in range(len(l)-n+1):
        res.append(tuple(l[i:i+n]))
    return res

In [30]:
ctoi = {chr(ord('a') + i): i+1 for i in range(26)}
ctoi['.'] = 0

In [31]:
itoc = {v: k for k,v in ctoi.items()}

In [39]:
xs, ys = [], []
for s in names:
    for ngram in build_ngram(s, 2):
        x = ngram[0]
        y = ngram[1]
        xs.append(ctoi[x])
        ys.append(ctoi[y])
xs, ys = torch.tensor(xs), torch.tensor(ys)
num = xs.nelement()

In [40]:
xenc = F.one_hot(xs, num_classes=27).float()

In [41]:
W = torch.randn((27, 27), requires_grad=True)

In [62]:
for k in range(500):
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    if k % 100 == 0:
        print(loss.item())
    W.grad = None
    loss.backward()
    W.data += -50 * W.grad

2.480508327484131
2.480471134185791
2.480448007583618
2.480433225631714
2.4804229736328125


In [61]:
for i in range(5):
    out = []
    ix = 0
    while True:
        xenc_in = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc_in @ W
        counts = logits.exp()
        probs = counts / counts.sum(1, keepdims=True)
        ix = torch.multinomial(probs, num_samples=1, replacement=True).item()
        out.append(itoc[ix])
        if ix == 0:
            break
    print("".join(out))

drete.
gidenioni.
hipaliernamoseeradeeri.
ph.
taquleneiayxyn.
